In [1]:
import pandas as pd
import numpy as np
import torch

In [3]:
df = pd.read_csv("data/transformed/df.csv")
reviews_scraped = pd.read_csv("data/transformed/reviews_scraped.csv")
products_scraped = pd.read_csv("data/transformed/products_scraped.csv")

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
class OurModelParameters():
    def __init__(self):
        pass

class OurModel(torch.nn.Module):

    def __init__(self, parameters: OurModelParameters):
        pass

    def forward(self, row_in_fake_real_products_df):
        """"
            row_in_fake_real_products_df is a df with the following columns:
                - product_title
                - review_title
                - rating1, rating2, rating3, rating4, rating5 (the percentages of ratings from each brackets, as values between 0.0 and 1.0)
                - 
        """
        pass
        

In [ ]:
from transformers import BertTokenizerFast, Automodel

bert = AutoModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

In [4]:
def tokenize_ds(train, val, test, prop = 'REVIEW_TEXT'):
    #This function takes one of the text properties of the dataset and produces an object of
    # type transformers.tokenization_utils_base.BatchEncoding, which is then used for producing
    # attention masks.

    max_seq_len = 512
        
    # tokenize and encode sequences in the training set
    tokens_train_review = tokenizer.batch_encode_plus(
        train[prop].tolist(),
        max_length = max_seq_len,
        pad_to_max_length=True,
        truncation=True,
        return_token_type_ids=False
    )

    # tokenize and encode sequences in the validation set
    tokens_val_review = tokenizer.batch_encode_plus(
        val[prop].tolist(),
        max_length = max_seq_len,
        pad_to_max_length=True,
        truncation=True,
        return_token_type_ids=False
    )

    # tokenize and encode sequences in the test set
    tokens_test_review = tokenizer.batch_encode_plus(
        test[prop].tolist(),
        max_length = max_seq_len,
        pad_to_max_length=True,
        truncation=True,
        return_token_type_ids=False
    )

    return tokens_train_review, tokens_val_review, tokens_test_review
